## 3. NCC test
Identify causal relations on MSCOCO dataset

### 3-1. MSCOCO dataset
subset of 99,309 MSCOCO images belonging to 20 Pascal object categories. 
resize (224x224, 짧은 쪽이 224 pixel로 rescale하고 224x224 centercrop).

In [ ]:
import json
import numpy as np
import pylab
import matplotlib.pyplot as plt
import skimage.io as io
from PIL import Image
from pycocotools.coco import COCO
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from torchvision.utils import draw_bounding_boxes
from torchvision.io import read_image

In [ ]:
# object categories in pascal voc
categories = [
    'airplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 
    'dining table', 'dog', 'horse', 'motorcycle', 'person', 'potted plant', 'sheep', 'couch', 'train', 'tv'
]
# 16 matched categories in coco dataset
# no matched in coco: aeroplane, motorbike, sofa, television
# aeroplane - airplane, motorbike - motorcycle, sofa - couch, television - tv

In [ ]:
dataDir = 'data/coco'
dataType='train2014' #'val2014'
annFile='{}/annotations/instances_{}.json'.format(dataDir, dataType)
# initialize COCO api for instance annotations
coco=COCO(annFile)
# categories
cats = coco.loadCats(coco.getCatIds())
coco_cats = {cat['id']: cat['name'] for cat in cats}

# get all images containing given categories, select one at random
idx = 0
dataset = []
for cat in categories:
    catIds = coco.getCatIds(catNms=cat)
    imgIds = coco.getImgIds(catIds=catIds)
    print(f'{cat}: {len(imgIds)}')
    for imgId in imgIds:
        annIds = coco.getAnnIds(imgIds=imgId, catIds=catIds, iscrowd=None)
        anns = coco.loadAnns(annIds)
        label = {'imgId': imgId, 'categories': [], 'bbox': []}
        for i, ann in enumerate(anns):
            label['categories'].append(coco_cats[ann['category_id']])
            label['bbox'].append(ann['bbox'])
        
        img_file_name = coco.loadImgs(imgId)[0]['file_name']
        label['file_name'] = img_file_name
        
        # dataset[idx] = label
        dataset.append(label)
        idx += 1
print(f'Number of images: {len(dataset)}')

# dataType1 = dataType.reshape('2014','')
# with open(f'data/coco/{dataType1}Ids.txt', 'w') as f:
#     json.dump(dataset, f, indent=4)

In [ ]:
def show_img(imgid, category):
    # load and display instance annotations
    fig, ax = plt.subplots(figsize=(10, 8))
    # load image
    img = coco.loadImgs(imgid)[0]
    I = io.imread(img['coco_url'])
    ax.imshow(I); plt.axis('off')
    # load annotations
    annIds = coco.getAnnIds(imgIds=img['id'], 
                            catIds=coco.getCatIds(catNms=category), 
                            iscrowd=None)
    anns = coco.loadAnns(annIds)
    coco.showAnns(anns, draw_bbox=True)
    for i, ann in enumerate(anns):
        ax.text(anns[i]['bbox'][0], anns[i]['bbox'][1], coco_cats[anns[i]['category_id']], style='italic', 
                bbox={'facecolor': 'white', 'alpha': 0.7, 'pad': 5})

In [ ]:
show_img(393221, 'airplane')

In [ ]:
# MSCOCO image (bbox + label)
class COCODataset(Dataset):
    def __init__(self, dataDir='data/coco', transform=None):
        # dataDir = dataDir
        dataType='train2014'
        # annFile='{}/annotations/instances_{}.json'.format(dataDir, dataType)
        self.img_path = f'{dataDir}/{dataType}/'

        # initialize COCO api for instance annotations
        # self.coco=COCO(annFile)

        # json
        with open('data/coco/trainIds.txt', 'r') as f:
            self.dataset = json.load(f) # 'img_id', 'bbox', 'anns'
        
        # transform
        self.transform = transform

    def __getitem__(self, index):
        data = self.dataset[index]
        # img = Image.open(self.img_path+data['file_name'])
        object_img = read_image(self.img_path+data['file_name'])
        context_img = read_image(self.img_path+data['file_name'])
        
        cats = torch.tensor([categories.index(x) for x in data['categories']])
        
        # img = draw_bounding_boxes(img, bboxs)
        mask = torch.zeros(img.shape[1:], dtype=int)
        for bbox in data['bbox']:
            x1, x2 = int(bbox[1]), int(bbox[1]+bbox[3])+1
            y1, y2 = int(bbox[0]), int(bbox[0]+bbox[2])+1
            mask[x1:x2,y1:y2] = 1

        mask = torch.tensor(mask>0, dtype=torch.uint8)
        
        objectImg = torch.where(mask>0, img, mask)
        contextImg = torch.where(mask>0, mask, img)
        
        objectImg = T.ToPILImage()(objectImg)
        contextImg = T.ToPILImage()(contextImg)
                
        if self.transform is not None:
            objectImg = self.transform(objectImg)
            contextImg = self.transform(contextImg)

        return objectImg, contextImg, cats
    
    def __len__(self):
        return len(self.dataset)

In [ ]:
transform = T.Compose([
    # All images are rescaled to ensure that 
    # their shorter side is 224 pixels long, 
    # then cropped to the central 224×224 square
    T.Resize(224),
    T.CenterCrop(224),
    T.ToTensor()
])
dataset = COCODataset(transform=transform)
dataloader = DataLoader(dataset, batch_size=1)

### 3-2. Model for object features

In [ ]:
def feature_extractor():
    model_ft = resnet18(pretrained=True)
    # finetune
    for param in model_ft.parameters():
        param.requires_grad = False
    # modify classifier
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Flatten()
    # features = model_ft._modules.get('avgpool')
    return model_ft

In [ ]:
class Classifier(nn.Module):
    #  train on Pascal VOC 2012 dataset!!
    def __init__(self, in_features=512, hidden_dim=512):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(in_features, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, 20)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.classifier(x)
        # softmax
        return x

In [ ]:
device = 'cpu'

resnet = feature_extractor().to(device)

classifier = Classifier()
classifier.load_state_dict(torch.load('results/object_classifier.pt'))
classifier = classifier.to(device)

### 3-3. Test

In [ ]:
for objectImg, contextImg, cats in dataloader:
    # plt.imshow(img1[0].permute(1,2,0))
    # plt.show()
    o_features = resnet(objectImg)
    c_features = resnet(contextImg)
    
    o_logodds = classifier(o_features)
    c_logodds = classifier(c_features)
    
    pair = (o_features, o_logodds)
    break